This notebook fits the diffraction geometry + module geometry to a series of powder patterns (with different beam centers)

In [10]:
# %matplotlib widget
import matplotlib.pyplot as pl
from datetime import datetime
import time
start = time.time()
import numpy as np, os
print(np.__version__)
from ImageD11 import transform
from ImageD11.transformer import transformer
from ImageD11.parameters import read_par_file
from ImageD11.columnfile import columnfile
import h5py
import scipy.optimize
import fabio
%matplotlib inline

1.23.5


In [11]:
wl = 0.9762535309700807
12.3985/12.700082106417474 # keV : approximate wavelength
12.3985/wl # Å : approximate energy


12.700082106417474

In [12]:
#   Creat par files from PONI file pyFAI
    

pixel1 =  7.4999999999999995e-05
pixel2 = 7.4999999999999995e-05
Wavelength = 9.762535309700807e-11

# par/1.par
Distance = 0.1469669042968765
Poni1 = 0.15888894388184904
Poni2 = 0.15730019810691892
Rot1 = 0.0016449807968716723
Rot2 = 0.0007732509695985323
Rot3 = 1.4834851722994806e-06


# par/1.par
# Distance = 0.146985549292152
# Poni1 = 0.15887909767411623
# Poni2 = 0.11720502785985053
# Rot1 = 0.0011568701681032478
# Rot2 = 0.0007863706042899544
# Rot3 = 2.937075750053691e-10


# par/2.par
# Distance = 0.14700536763721697
# Poni1 = 0.19936408682389326
# Poni2 = 0.11722191471270342
# Rot1 = 0.0012135697775101859
# Rot2 = 0.0010562058484545475
# Rot3 = -2.4374960375501512e-09


# par/3.par
# Distance = 0.1470438827119443
# Poni1 = 0.23930389773515182
# Poni2 = 0.11721099149214317
# Rot1 = 0.0011045084838636268
# Rot2 = 0.0012472885814468783
# Rot3 = -6.940738623584151e-10


# par/4.par
# Distance = 0.14702122160296238
# Poni1 = 0.2393194355972989
# Poni2 = 0.1573433827509926
# Rot1 = 0.0017515329654952832
# Rot2 = 0.0012620019424374772
# Rot3 = 1.7588218803788978e-07


# par/5.par
# Distance = 0.14698380738892386
# Poni1 = 0.19937742723728372
# Poni2 = 0.15732383533080238
# Rot1 = 0.001700416800773225
# Rot2 = 0.0010709132087503377
# Rot3 = 8.721689967354811e-07


cos_tilt = np.cos(Rot1) * np.cos(Rot2)

sin_tilt = np.sqrt(1.0 - cos_tilt * cos_tilt)
tan_tilt = sin_tilt / cos_tilt

# This is tilt plane rotation
if sin_tilt == 0:
    # tilt plan rotation is undefined when there is no tilt!, does not matter
    cos_tilt = 1.0
    sin_tilt = 0.0
    cos_tpr = 1.0
    sin_tpr = 0.0

else:
    cos_tpr = max(-1.0, min(1.0, -np.cos(Rot2) * np.sin(Rot1) / sin_tilt))
    sin_tpr = np.sin(Rot2) / sin_tilt
# print(Distance , cos_tilt)
directDist = 1.0e3 * Distance / cos_tilt
tilt = np.degrees(np.arccos(cos_tilt))
if sin_tpr < 0:
    tpr = -np.degrees(np.arccos(cos_tpr))
else:
    tpr = np.degrees(np.arccos(cos_tpr))

centerX = (Poni2 + Distance * tan_tilt * cos_tpr) / pixel2
if abs(tilt) < 1e-5:  # in degree
    centerY = (Poni1) / detector.pixel1
else:
    centerY = (Poni1 + Distance * tan_tilt * sin_tpr) / pixel1

print('cell__a', 4.15682600)
print('cell__b', 4.15682600)
print('cell__c', 4.15682600)
print('cell_alpha', 90.0)
print('cell_beta', 90.0)
print('cell_gamma', 90.0)
print('cell_lattice_[P,A,B,C,I,F,R]', 'P')
print('chi', 0.0)
print('distance', directDist)
print('fit_tolerance', 0.025)
print('min_bin_prob', 1e-05)
print('no_bins', 10000)
print('o11', 1)
print('o12', 0)
print('o21', 0)
print('o22', -1)
print('omegasign', 1.0)
print('t_x', 0)
print('t_y', 0)
print('t_z', 0)
print('tilt_x', Rot3)
print('tilt_y', Rot2)
print('tilt_z', -Rot1)
print('wavelength', Wavelength * 1.0e10)
print('wedge', 0.0)
print('weight_hist_intensities', 0)
print('y_center', centerX)
print('y_size', 0.075)
print('z_center', centerY)
print('z_size', 0.075)

cell__a 4.156826
cell__b 4.156826
cell__c 4.156826
cell_alpha 90.0
cell_beta 90.0
cell_gamma 90.0
cell_lattice_[P,A,B,C,I,F,R] P
chi 0.0
distance 146.96714707759696
fit_tolerance 0.025
min_bin_prob 1e-05
no_bins 10000
o11 1
o12 0
o21 0
o22 -1
omegasign 1.0
t_x 0
t_y 0
t_z 0
tilt_x 1.4834851722994806e-06
tilt_y 0.0007732509695985323
tilt_z -0.0016449807968716723
wavelength 0.9762535309700807
wedge 0.0
weight_hist_intensities 0
y_center 2094.1125353801635
y_size 0.075
z_center 2120.0344847933484
z_size 0.075


In [14]:
# choose the frames from 6 frames
# frms = [1,2,3,4,5,6]
frms = [1,2,3,4]
frms

[1, 2, 3, 4]

In [15]:
# created via moduleSpatialPeakSearch
f, ((a3, a4),(a1,a2)) = pl.subplots(2,2,figsize=(10,10))
cf = {}
a0 = 4.15682600
ds0 = a0*a0
for i in frms:
    # Load the peaks and parameters:
    c = columnfile( 'eigerSpatial/flt/%d.flt'%(i) )
    print('-----pks-----', c.sc.shape)
    c.parameters.loadparameters('eigerSpatial/par/%d.par'%(i) )
    c.updateGeometry()
    # Create column of frame number
    c.addcolumn(np.ones(c.nrows)*i,'frame')
    # (d*_obs)^2 / (a_0)^2 from cubic hkl to d-spacing formula
    c.addcolumn(np.ones(c.nrows)*c.ds**2*ds0,'hkl2')
    # nearest integers == fit target values
    c.addcolumn(np.round(c.hkl2),'ihkl2')
    # something like a strain
    se = abs( c.ihkl2 - c.hkl2 )/np.where( c.ihkl2 > 0, c.ihkl2, 1)
    a3.plot(c.hkl2, se, ",")
    a4.plot(c.hkl2, c.eta, ",")
    c.filter((c.Number_of_pixels>7) & (c.ihkl2>0) &  (se<6e-3) )
    print('---filtered---', c.sc.shape)
    # c.filter((c.Number_of_pixels>7) & (c.ihkl2>2) &  (se<2e-3) ) # original
    a1.plot(c.fc,c.sc,',')
    a2.plot(c.hkl2,c.eta,',')
    cf[i] = c
    print('np.unique(c.ihkl2)', np.unique(c.ihkl2))
    print()
    uihkl2 = np.unique(c.ihkl2)
a1.set(title='Peaks on detector', ylabel='sc', xlabel='fc')
a2.set(title='Selected peaks', ylabel='azimuth / deg', xlabel='$H^2$')
a3.set(yscale='log', ylim=(1e-6,1), title='relative error', ylabel='$\Delta H^2/H^2$', xlabel='$H^2$')
_ = a4.set(title='All peaks', ylabel='azimuth / deg', xlabel='$H^2$')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-----pks----- (114680,)
---filtered--- (109146,)
np.unique(c.ihkl2) [ 1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 13. 14. 16.]

-----pks----- (110382,)
---filtered--- (104671,)
np.unique(c.ihkl2) [ 1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 13. 14. 16. 17. 18.]

-----pks----- (107923,)
---filtered--- (102067,)
np.unique(c.ihkl2) [ 1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 13. 14. 16. 17. 18. 19.]

-----pks----- (104330,)
---filtered--- (96087,)
np.unique(c.ihkl2) [ 1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 13. 14. 16. 17. 18. 19. 20.]



In [16]:
# put together all of the peaks:
allpks = np.concatenate( [cf[f].bigarray for f in frms], axis=1)
# ... and place them into an ImageD11 colummfile object
allc = c.copy()
allc.set_bigarray(allpks)
allc.nrows, np.sum([cf[f].nrows for f in frms])

(411971, 411971)

In [17]:
# Remove "bad" peaks from virtual shared pixels
# for col in 255.5, 513.5, 771.5, 1295.5, 1553.5, 1811.5:
#     allc.filter(abs(allc.fc-col)>2.1)
# for row in 255.5, 805.5, 1355.5, 1905.5:
#     allc.filter(abs(allc.sc-row)>2.1)
pl.figure(2)
print(allc.fc.shape)
print(allc.sc.shape)
print(np.linalg.norm(allc.ihkl2-allc.hkl2))
# print(allc,allc.fc,allc.ihkl2, allc.hkl2)
pl.plot(allc.fc,allc.ihkl2-allc.hkl2, ',', alpha=0.2)
pl.figure(3)
pl.plot(allc.sc,allc.ihkl2-allc.hkl2, ',', alpha=0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(411971,)
(411971,)
3.6269994484182595


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
pl.figure()
pl.plot(allc.hkl2,allc.eta,',')
pl.plot(allc.ihkl2,allc.eta,',')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
frm = np.unique(allc.frame)
modu = np.unique(allc.module)
print(frm, modu)

[1. 2. 3. 4.] [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]


In [20]:
# set up a fitting problem : these parameters will be fed to the calchkl2 routine below

# distance, tilts, centers for each image 
pars = [ ]
pnames = [ ]
for f in frm:
    pars.append( cf[f].parameters.get('distance') / a0 ) # distance scaled on lattice
    pnames.append( '%d:distance/a'%(f) )
    for pname in 'y_center z_center tilt_y tilt_z'.split():
        pars.append( cf[f].parameters.get(pname) )
        pnames.append( '%d:%s'%(f, pname) )

pars.append( a0 ) # effectively the wavelength
pnames.append( 'a0' )
framepars = len(pars)
nframes = len(cf.keys())
pars += [0. for i in range(31*3)] # 32 module centers and tilts, BUT LAST IS FIXED
pnames += ["M[%d,%d]"%(i,j) for i in range(31) for j in range(3) ]
print()
print()
for n,v in zip(pnames, pars):
    print(n,v)
# offset, rotation for each module



1:distance/a 35.35561678010986
1:y_center 2094.1125353801635
1:z_center 2120.0344847933484
1:tilt_y 0.0007732509695985323
1:tilt_z -0.0016449807968716723
2:distance/a 35.36007836208887
2:y_center 1560.4664611583082
2:z_center 2119.9291052067574
2:tilt_y 0.0007863706042899544
2:tilt_z -0.0011568701681032478
3:distance/a 35.36485719778139
3:y_center 1560.5801780510244
3:z_center 2660.2580656669206
3:tilt_y 0.0010562058484545475
3:tilt_z -0.0012135697775101859
4:distance/a 35.37412602418738
4:y_center 1560.6477361352484
4:z_center 3193.1640546401663
4:tilt_y 0.0012472885814468783
4:tilt_z -0.0011045084838636268
a0 4.156826
M[0,0] 0.0
M[0,1] 0.0
M[0,2] 0.0
M[1,0] 0.0
M[1,1] 0.0
M[1,2] 0.0
M[2,0] 0.0
M[2,1] 0.0
M[2,2] 0.0
M[3,0] 0.0
M[3,1] 0.0
M[3,2] 0.0
M[4,0] 0.0
M[4,1] 0.0
M[4,2] 0.0
M[5,0] 0.0
M[5,1] 0.0
M[5,2] 0.0
M[6,0] 0.0
M[6,1] 0.0
M[6,2] 0.0
M[7,0] 0.0
M[7,1] 0.0
M[7,2] 0.0
M[8,0] 0.0
M[8,1] 0.0
M[8,2] 0.0
M[9,0] 0.0
M[9,1] 0.0
M[9,2] 0.0
M[10,0] 0.0
M[10,1] 0.0
M[10,2] 0.0
M[11

In [21]:
def module(a,i,j):
    #p = padding around the outside for debug
    fs = 1030
    ss = 514
    fo = [ 0,  10,   20,   30]
    so = [ 0, 37, 2*37, 37*3, 37*4, 37*5, 37*6, 37*7]
    f = fs*j + fo[j]
    s = ss*i + so[i]
    return a[s:s+ss,f:f+fs]

def module_center( i, j ):
    fs = 1030
    ss = 514
    fo = np.array([ 0,  10,   20,   30])
    so = np.array([ 0, 37, 2*37, 37*3, 37*4, 37*5, 37*6, 37*7])
    return so[i] + ss*(i+0.5), fo[j] + fs*(j+0.5)

def mp31_32( p31 ):
    """ Compute the parameters of module 16 from the other 15 
    We set the sum of all 16 to be zero
    """
    p32 = np.array(list(p31) + [0,0,0]).reshape( (32, 3) ) 
    p32[-1,:] = -np.sum(p32, axis=0) # last module is fixed to make mean zero
    # print('p32.shape',p32.shape)
    return p32
    
    
def do_spatial(s_raw, f_raw, m, modpars):
    """
    s_raw = co-ordinate in the slow pixel direction
    f_raw = co-ordinate in the fast pixel direction
    m = module for this peak (in the range 0,31 for Eiger16M)
    modpars = spatial parameters (offset_slow, offset_fast, angle_rad)
    """
    j = m // 8
    i = m % 8
    cs, cf = module_center( i, j )
    shift_s = np.array( modpars[:,0] )
    shift_f = np.array( modpars[:,1] )
    ds = s_raw[:] - cs + shift_s[m]
    df = f_raw[:] - cf + shift_f[m]
    rotates = modpars[:,2]
    sinr = np.sin(rotates)
    cosr = np.cos(rotates)
    dsr =  cosr[m] * ds  + sinr[m] * df
    dfr = -sinr[m] * ds  + cosr[m] * df
    sc = dsr + cs
    fc = dfr + cf
    return sc,fc    

def calchkl2( colf, *args ):
    """
    colf = columnfile (table) containing labelled peaks
    *args = parameters to be fitted
    """
    fpars = np.array(list(args))
    global framepars, frm # above
    modpars = mp31_32(fpars[framepars:]) # make last module == sum of previous
    ycalc = np.zeros(colf.nrows, float)
    # apply the current spatial correction
    m = colf.module[:].astype(int)
    
    sc, fc = do_spatial( colf.s_raw, colf.f_raw, m, modpars )
    w = 0.9762535309700807
    a0 = fpars[framepars-1]
    for k,rf in enumerate(frm):
        f = int(rf)
        lo = np.searchsorted( colf.frame, f )   # assume data still sorted by frame
        hi = np.searchsorted( colf.frame, f+1 )
        # compute x,y,z laboratory
        x,y,z = transform.compute_xyz_lab( (sc[lo:hi], fc[lo:hi]), 
                                               distance = fpars[k*5]*a0, # take out ds0 again
                                               y_center = fpars[k*5+1],
                                               z_center = fpars[k*5+2],
                                               y_size=0.075, z_size=0.075,
                                               tilt_y = fpars[k*5+3],
                                               tilt_z = fpars[k*5+4],
                                               tilt_x = 0,
                                               o11=1,o12=0,o21=0,o22=-1 )
        # radius:
        r = np.sqrt(y*y + z*z)
        # two theta:
        tthrad = np.arctan2(r, x)
        # wvln = 2*d*sin(th)
        # 1/d = 2*sin(th)/wvln
        # c.ds**2*ds0
        ds = 2*np.sin( tthrad/2 ) / w
        ycalc[lo:hi] = ds*ds*a0*a0
    scor = np.mean((ycalc - colf.ihkl2)**2)
    strain = (ycalc - colf.ihkl2)/colf.ihkl2  # approximately zero
    return strain

In [22]:
print('allc.shape', allc.sc.shape)
yc = calchkl2( allc, *pars )
print('yc.shape', yc.shape)


pl.figure()
pl.plot(allc.eta, yc*allc.ihkl2+allc.ihkl2, ",")
pl.plot(allc.eta,allc.ihkl2,',')
pl.figure()
pl.hist(yc,bins=1024)
pl.semilogy()

allc.shape (411971,)
yc.shape (411971,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [23]:
avgi = allc.sum_intensity / allc.Number_of_pixels   # average intensity in a peak 
# sigma = avgi/avgi.max()                             # relative weights
sigma = np.sqrt(avgi/avgi.max())

# print(np.max(avgi))
# print(np.min(sigma))
tic = time.time()
fpars, matrx = scipy.optimize.curve_fit( calchkl2,               # function to fit
                               allc,                    # 'x' argument to function 
                               np.zeros(allc.nrows),    # y_obs values (integer hkls)  
                               pars,                    # parameters to be fitted
                               sigma = sigma  )

print('time', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('toc', time.time()-tic)

time 2023-09-27 12:13:47
toc 41.09238362312317


In [24]:
#print(fpars, matrx)
from datetime import datetime
pl.figure()
corr = matrx.copy()
ser = np.sqrt(np.diag(corr))
corr = corr / ser 
corr = corr.T / ser
pl.imshow(corr, cmap='gist_ncar', vmin=-1)
pl.colorbar()
pl.title("correlation matrix")
print("Correlation above 80%")
for i in range(corr.shape[0]):
    for j in range(i-1):
        if abs(corr[i,j])>0.8:
            print(pnames[i],pnames[j],corr[i,j])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Correlation above 80%
2:distance/a 1:distance/a 0.9331421967696724
2:z_center 1:z_center 0.8153178241246736
3:distance/a 1:distance/a 0.8875309056369072
3:distance/a 2:distance/a 0.9271630152879808
3:z_center 2:z_center 0.8741884135709709
4:distance/a 3:distance/a 0.8620740162016977
4:y_center 3:y_center 0.8425158963275029
4:z_center 3:z_center 0.8961237234834956
a0 1:distance/a 0.8075857980080123
M[1,2] M[1,0] -0.87873257474843
M[2,2] M[2,0] -0.8972115415682137
M[3,2] M[3,0] -0.9108017218726308
M[5,2] M[5,0] -0.8650409915153768
M[6,2] M[6,0] -0.9008423188247624
M[7,2] M[7,0] -0.8819823471814835
M[9,0] 2:z_center 0.8223839064038068
M[13,0] 2:z_center 0.8579556424854884
M[13,0] 3:z_center 0.8027754187855024
M[13,0] M[9,0] 0.826271625881738
M[15,2] M[15,0] -0.9107647174546952
M[16,2] M[16,0] 0.9240312122697644
M[20,2] M[20,0] 0.8913438340710934
M[21,1] 4:y_center 0.8441386632941699
M[24,2] M[24,0] 0.9408800140987156
M[25,2] M[25,0] 0.9557319114718621
M[26,2] M[26,0] 0.8561227039264842
M[

In [25]:
ycalc = calchkl2(allc, *fpars)
pl.figure()
pl.plot(allc.eta, ycalc*allc.ihkl2+allc.ihkl2, ",")
pl.plot(allc.eta,allc.ihkl2,',')
pl.figure()
pl.plot(allc.eta, ycalc, ",")
pl.figure()
pl.hist( ycalc, bins=1024)
pl.semilogy()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [26]:
allc.filter( abs(ycalc)<.0003 )


print('allc.shape', allc.sc.shape)
print('ycalc.shape', ycalc.shape)
print('type(ycalc)', type(ycalc))

allc.shape (367996,)
ycalc.shape (411971,)
type(ycalc) <class 'numpy.ndarray'>


In [27]:
ycalc = calchkl2(allc, *fpars)
pl.figure()
pl.plot(allc.Number_of_pixels,ycalc,'.', alpha=0.01)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
# repeat fitting with cleaned data
p2 = [p for p in fpars]  
tic = time.time()
fpars2, matrx2 = scipy.optimize.curve_fit( calchkl2, allc, np.zeros(allc.nrows), p2 )
print('time', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('toc', time.time()-tic)

time 2023-09-27 12:15:11
toc 34.901761054992676


In [29]:
ycalc = calchkl2(allc, *fpars2)
pl.figure()
pl.plot(allc.eta, ycalc*allc.ihkl2+allc.ihkl2, ",")
# pl.ylim(9.98, 10.02)
pl.figure()
pl.plot(allc.eta, ycalc, ",")
pl.figure()
pl.hist( ycalc, bins=1024)
pl.semilogy()

p16 = mp31_32(fpars2[framepars:])
print(p16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 6.58187003e-02 -3.31679418e-01 -1.14923596e-03]
 [-2.33867683e-01  9.99911160e-01 -9.33194934e-04]
 [-1.70005934e+00  5.92359396e-01 -7.05117236e-05]
 [-7.07122845e+00 -1.71763658e+00  2.50340945e-03]
 [ 3.33918095e-01  2.42655654e+00 -2.12527325e-03]
 [ 6.02799416e-02  2.80312182e+00 -9.05860773e-04]
 [-5.09481659e+00 -5.51734043e+00  2.19054745e-03]
 [-3.73162608e+00 -3.53037099e+00  1.44701046e-03]
 [-6.35202961e-01 -1.18281385e+00 -1.43065131e-03]
 [ 6.61427378e-02 -2.54170123e-01 -4.23147410e-04]
 [-4.56141752e-01  1.08223926e-01  5.44594909e-04]
 [-4.76627572e+00 -8.45146694e-01  3.11239887e-03]
 [-1.99365220e-01  4.07648310e-01 -8.62311831e-04]
 [ 4.42929334e-01  2.95817184e-01 -1.14061609e-04]
 [ 2.29014032e-01  2.27443377e-01  1.47265997e-04]
 [-3.74139155e-01 -2.25020901e-01  4.20327342e-04]
 [ 5.73223327e-01  4.89906259e-02 -2.61414718e-05]
 [ 6.51382129e-01 -2.12313875e-01 -2.11557330e-04]
 [ 8.48793495e-01  1.42968393e-01 -1.85893640e-04]
 [ 2.37389234e-02  8.33328313e-

In [30]:
pl.figure()
for i in frms:
    ycalc = calchkl2(cf[i], *fpars2)
    if i == 1:
        pl.plot(cf[i].eta,cf[i].ihkl2,'_',markersize=1, color='k', label='ihkl(210) ')
        pl.plot(cf[i].eta,cf[i].hkl2,'.',markersize=1, label='hkl (210) before correction')
    pl.plot(cf[i].eta,cf[i].ihkl2,'_',markersize=1)
    pl.plot(cf[i].eta,cf[i].hkl2,'.',markersize=1)
pl.xlabel('Azimuth (degree)')
pl.ylabel('$H^2$')
pl.legend(loc='upper right')
pl.ylim(0.996, 1.004)
pl.ylim(4.985, 5.015)


pl.figure()
for i in frms:
    ycalc = calchkl2(cf[i], *fpars2)
    if i == 1:
        pl.plot(cf[i].eta,cf[i].ihkl2,'.',markersize=1, label='ihkl(210) ')
        pl.plot(cf[i].eta, ycalc*cf[i].ihkl2+cf[i].ihkl2, ".",markersize=1, label='hkl (210) after correction')
    pl.plot(cf[i].eta,cf[i].ihkl2,'.',markersize=1)
    pl.plot(cf[i].eta, ycalc*cf[i].ihkl2+cf[i].ihkl2, ".",markersize=1)
pl.xlabel('Azimuth (degree)')
pl.ylabel('$H^2$')
pl.legend(loc='upper right')
pl.ylim(0.996, 1.004)
pl.ylim(4.985, 5.015)



pl.figure()
for i in frms:
    ycalc = calchkl2(cf[i], *fpars2)
    if i == 1:
        pl.plot(cf[i].eta,cf[i].ihkl2,'_',markersize=1, color='k', label='ihkl (100)')
        pl.plot(cf[i].eta,cf[i].hkl2,'.',markersize=1, label='hkl (100) before correction')
    pl.plot(cf[i].eta,cf[i].ihkl2,'_',markersize=1)
    pl.plot(cf[i].eta,cf[i].hkl2,'.',markersize=1)
pl.xlabel('Azimuth (degree)')
pl.ylabel('$H^2$')
pl.legend(loc='upper right')
pl.ylim(0.996, 1.004)
pl.ylim(0.985, 1.015)


pl.figure()
for i in frms:
    ycalc = calchkl2(cf[i], *fpars2)
    if i == 1:
        pl.plot(cf[i].eta,cf[i].ihkl2,'.',markersize=1, label='ihkl (100)')
        pl.plot(cf[i].eta, ycalc*cf[i].ihkl2+cf[i].ihkl2, ".",markersize=1, label='hkl (100) after correction')
    pl.plot(cf[i].eta,cf[i].ihkl2,'.',markersize=1)
    pl.plot(cf[i].eta, ycalc*cf[i].ihkl2+cf[i].ihkl2, ".",markersize=1)
pl.xlabel('Azimuth (degree)')
pl.ylabel('$H^2$')
pl.legend(loc='upper right')
pl.ylim(0.996, 1.004)
pl.ylim(0.985, 1.015)



np.savetxt("fitted_eiger_pars2.dat",fpars2)
np.savetxt("fitted_eiger_framepars2.dat",p16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
# Write spatially corrected files
a0 = fpars2[framepars-1]
for i in frms:
    p = read_par_file("eigerSpatial/par/%d.par"%(i))
    k = i-1
    p.parameters.update( {'distance' : fpars2[k*5]*a0,
                          'y_center' : fpars2[k*5+1],
                          'z_center' : fpars2[k*5+2],
                          'tilt_y' : fpars2[k*5+3],
                          'tilt_z' : fpars2[k*5+4] } )
    p.saveparameters("eigerSpatial/par/%ds.par"%(i))


for n,v in zip(pnames, fpars2):
    print(n,v)

1:distance/a 35.377574616605095
1:y_center 2093.8549347571666
1:z_center 2120.107550618203
1:tilt_y 0.0006192328693350761
1:tilt_z -0.0016862205129217244
2:distance/a 35.38358504792913
2:y_center 1560.2052163817698
2:z_center 2119.9786713647554
2:tilt_y 0.00059720317604637
2:tilt_z -0.0012439505542631824
3:distance/a 35.38597615242413
3:y_center 1560.286280353019
3:z_center 2660.8555293004642
3:tilt_y 0.0007229684713541034
3:tilt_z -0.0012357626459110944
4:distance/a 35.39439947584305
4:y_center 1560.3229108048333
4:z_center 3194.2307388923277
4:tilt_y 0.000835621250725883
4:tilt_z -0.0012061979326253843
a0 4.158237583036333
M[0,0] 0.06581870030184389
M[0,1] -0.33167941841419263
M[0,2] -0.0011492359581567028
M[1,0] -0.2338676827512559
M[1,1] 0.9999111604374242
M[1,2] -0.0009331949340618836
M[2,0] -1.7000593361439595
M[2,1] 0.5923593964650528
M[2,2] -7.051172361749558e-05
M[3,0] -7.071228446219838
M[3,1] -1.717636582223073
M[3,2] 0.0025034094483860686
M[4,0] 0.33391809464072697
M[4,1] 2

In [32]:
# Verify and create conventional distortion files
s, f = np.mgrid[0:4371,0:4150]

print(s.shape)
print(f.shape)
print(f)
print(s)
m = np.zeros(s.shape, np.int)-1
print(m.shape)
#    fs = 1030
#    ss = 514
#    fo = [ 0,  10,   20,   30]
#    so = [ 0, 37, 2*37, 37*3, 37*4, 37*5, 37*6, 37*7]
for i in range(8):
    for j in range(4):
        a = module(m,i,j)
        a[:,:] = i*4+j
# Make an image of which module is this pixel
# fill gaps: ... yes, this could be more elegant
m[514:514+37//2,:]=m[513,:]
m[514+37//2:514+37,:]=m[514+37+1,:]

m[514*2+37:514*2+37+37//2,:]=m[514*2+37-1,:]
m[514*2+37+37//2:514*2+37*2,:]=m[514*2+2*37+1,:]

m[514*3+37*2:514*3+37*2+37//2,:]=m[514*3+37*2-1,:]
m[514*3+37*2+37//2:514*3+37*3,:]=m[514*3+3*37+1,:]

m[514*4+37*3:514*4+37*3+37//2,:]=m[514*4+3*37-1,:]
m[514*4+37*3+37//2:514*4+37*4,:]=m[514*4+4*37+1,:]

m[514*5+37*4:514*5+37*4+37//2,:]=m[514*5+4*37-1,:]
m[514*5+37*4+37//2:514*5+37*5,:]=m[514*5+5*37+1,:]

m[514*6+37*5:514*6+37*5+37//2,:]=m[514*6+5*37-1,:]
m[514*6+37*5+37//2:514*6+37*6,:]=m[514*6+6*37+1,:]

m[514*7+37*6:514*7+37*6+37//2,:]=m[514*7+6*37-1,:]
m[514*7+37*6+37//2:514*7+37*7,:]=m[514*7+7*37+1,:]


m[:,1028:1034]=m[:,1025][:,np.newaxis]
m[:,1033:1040]=m[:,1041][:,np.newaxis]

m[:,2068:2074]=m[:,2065][:,np.newaxis]
m[:,2073:2080]=m[:,2081][:,np.newaxis]

m[:,3108:3114]=m[:,3105][:,np.newaxis]
m[:,3113:3120]=m[:,3211][:,np.newaxis]

# verify we did not mess up the real pixels:
for i in range(8):
    for j in range(4):
        a = module(m,i,j)
        assert((a[:,:] == i*4+j).all())
pl.figure()
pl.imshow(m)
pl.colorbar()

(4371, 4150)
(4371, 4150)
[[   0    1    2 ... 4147 4148 4149]
 [   0    1    2 ... 4147 4148 4149]
 [   0    1    2 ... 4147 4148 4149]
 ...
 [   0    1    2 ... 4147 4148 4149]
 [   0    1    2 ... 4147 4148 4149]
 [   0    1    2 ... 4147 4148 4149]]
[[   0    0    0 ...    0    0    0]
 [   1    1    1 ...    1    1    1]
 [   2    2    2 ...    2    2    2]
 ...
 [4368 4368 4368 ... 4368 4368 4368]
 [4369 4369 4369 ... 4369 4369 4369]
 [4370 4370 4370 ... 4370 4370 4370]]
(4371, 4150)


/local/slurmtmp.1482856/ipykernel_1239302/4157581939.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  m = np.zeros(s.shape, np.int)-1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
# full image spatial:
sc,fc = do_spatial(s.ravel(),f.ravel(),m.ravel(), p16)
sc.shape=s.shape
fc.shape=f.shape
ds = sc - s      # LUT
df = fc - f      # 

fabio.edfimage.edfimage((fc-f).astype(np.float32)).write("e2dx.edf")
fabio.edfimage.edfimage((sc-s).astype(np.float32)).write("e2dy.edf")





df = fabio.open("e2dx.edf").data
ds = fabio.open("e2dy.edf").data

In [35]:
tic = time.time()


t1 = transformer()

t1.loadfiltered("eigerSpatial/flt/1.flt")
c1 = t1.colfile
c1.sc[:] = ds[np.round( c1.s_raw[:].astype(int) ), np.round( c1.f_raw[:].astype(int) ) ] + c1.s_raw[:]
c1.fc[:] = df[np.round( c1.s_raw[:].astype(int) ), np.round( c1.f_raw[:].astype(int) ) ] + c1.f_raw[:]
t1.loadfileparameters("eigerSpatial/par/1s.par")
t1.addcellpeaks()
print(t1.theorytth)
print(t1.parameterobj.get_parameters())
print()
t1.compute_tth_eta()

print()
print('  1 ')
print()

t2 = transformer()
t2.loadfiltered("eigerSpatial/flt/2.flt")
c = t2.colfile
c.sc[:] = ds[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.s_raw[:]
c.fc[:] = df[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.f_raw[:]
t2.loadfileparameters("eigerSpatial/par/2s.par")
t2.compute_tth_eta()

print()
print('  2 ')
print()


t3 = transformer()
t3.loadfiltered("eigerSpatial/flt/3.flt")
c = t3.colfile
c.sc[:] = ds[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.s_raw[:]
c.fc[:] = df[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.f_raw[:]
t3.loadfileparameters("eigerSpatial/par/3s.par")
t3.compute_tth_eta()

print()
print('  3 ')
print()



t4 = transformer()
t4.loadfiltered("eigerSpatial/flt/4.flt")
c = t4.colfile
c.sc[:] = ds[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.s_raw[:]
c.fc[:] = df[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.f_raw[:]
t4.loadfileparameters("eigerSpatial/par/4s.par")
t4.compute_tth_eta()

print()
print('  4 ')
print()



# t5 = transformer()
# t5.loadfiltered("eigerSpatial/flt/5.flt")
# c = t5.colfile
# c.sc[:] = ds[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.s_raw[:]
# c.fc[:] = df[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.f_raw[:]
# t4.loadfileparameters("eigerSpatial/par/5s.par")
# t5.compute_tth_eta()

# print()
# print('  5 ')
# print()




# t6 = transformer()
# t6.loadfiltered("eigerSpatial/flt/6.flt")
# c = t6.colfile
# c.sc[:] = ds[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.s_raw[:]
# c.fc[:] = df[np.round( c.s_raw[:].astype(int) ), np.round( c.f_raw[:].astype(int) ) ] + c.f_raw[:]
# t6.loadfileparameters("eigerSpatial/par/6s.par")
# t6.compute_tth_eta()
# print()
# print('  6 ')
# print()


print('time', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('toc', time.time()-tic)


[13.48734885 19.11855669 23.47063384 27.16621263 30.44598974 33.43320967
 38.79692991 41.2540689  43.59646365 45.84228161 48.00565092 50.09782516
 52.12795019 56.03108225]
{'omegasign': 1.0, 'z_center': 2120.107550618203, 'y_center': 2093.8549347571666, 'distance': 147.10836036743947, 'z_size': 0.075, 'y_size': 0.075, 'tilt_z': -0.0016862205129217244, 'tilt_y': 0.0006192328693350761, 'tilt_x': 1.4834851722994806e-06, 'fit_tolerance': 0.025, 'wavelength': 0.9762535309700807, 'wedge': 0.0, 'chi': 0.0, 'cell__a': 4.156826, 'cell__b': 4.156826, 'cell__c': 4.156826, 'cell_alpha': 90.0, 'cell_beta': 90.0, 'cell_gamma': 90.0, 'cell_lattice_[P,A,B,C,I,F,R]': 'P', 'o11': 1, 'o12': 0, 'o21': 0, 'o22': -1, 't_x': 0, 't_y': 0, 't_z': 0, 'no_bins': 10000, 'min_bin_prob': 1e-05, 'weight_hist_intensities': 0}


  1 




  2 




  3 




  4 

time 2023-09-27 12:16:52
toc 1.1630425453186035


In [36]:

pl.figure()
pl.plot(t1.colfile.eta,t1.colfile.tth,",", label='Corrected data (hkl)')
pl.plot(t2.colfile.eta,t2.colfile.tth,",")
pl.plot(t3.colfile.eta,t3.colfile.tth,",")
pl.plot(t4.colfile.eta,t4.colfile.tth,",")
# pl.plot(t5.colfile.eta,t5.colfile.tth,",")
# pl.plot(t6.colfile.eta,t6.colfile.tth,",")
pl.xlabel('Azimuth (degree)')
pl.ylabel('Two Theta (degree)')
pl.legend(loc='upper right')
for tth in t1.theorytth:
    pl.hlines(y=tth, xmin=np.min(allc.eta), xmax=np.max(allc.eta), linewidth=1, color='k', linestyles='dashed', label='Calculated')
pl.xlim(np.min(allc.eta), np.max(allc.eta))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-179.9967161001967, 179.99353247315403)